In [1]:
import geopandas as gpd
import numpy as np
import xarray as xr
import rasterio
from rasterio import features

# --------------------------------------------------------
# 1) Read Longhurst shapefile
# --------------------------------------------------------
shp_path = "/proj/kimyy/Observation/longhurst_v4_2010/Longhurst_world_v4_2010.shp"
gdf = gpd.read_file(shp_path)

# Map province codes (e.g., WTRA, MONS) to integer IDs
prov_codes = gdf["ProvCode"].tolist()
prov_to_int = {code: i+1 for i, code in enumerate(prov_codes)}  # 1-based indexing
int_to_prov = {v: k for k, v in prov_to_int.items()}

# --------------------------------------------------------
# 2) Define output grid (1° × 1° global grid)
# --------------------------------------------------------
lon_res = 1.0
lat_res = 1.0

# lon: -180 → 179, step = 1
lons = np.arange(-180, 180, lon_res)

# lat: 90 → -89 (top to bottom, raster convention)
lats = np.arange(90, -90, -lat_res)

nlon = len(lons)
nlat = len(lats)

# Affine transform for the raster grid
transform = rasterio.transform.from_origin(-180, 90, lon_res, lat_res)

# --------------------------------------------------------
# 3) Rasterize polygons to integer mask (fast method)
# --------------------------------------------------------
# Each element: (geometry, value)
shapes = [
    (geom, prov_to_int[code])
    for geom, code in zip(gdf.geometry, gdf["ProvCode"])
]

# Rasterization: convert polygons to grid mask
raster = features.rasterize(
    shapes=shapes,
    out_shape=(nlat, nlon),
    transform=transform,
    fill=0,          # 0 = outside any Longhurst province
    dtype="int32"
)

# --------------------------------------------------------
# 4) Create xarray Dataset
# --------------------------------------------------------
ds = xr.Dataset(
    {
        "Longhurst_mask": (("lat", "lon"), raster)
    },
    coords={
        "lat": lats,
        "lon": lons,
    },
    attrs={
        "description": "Longhurst Biogeochemical Provinces (1-degree rasterized)",
        "source": "Longhurst_world_v4_2010.shp",
        "province_code_mapping": str(int_to_prov)
    }
)

# --------------------------------------------------------
# 5) Save to NetCDF
# --------------------------------------------------------
ds.to_netcdf("/proj/kimyy/Observation/longhurst_v4_2010/Longhurst_1deg_mask.nc")

print("Saved: /proj/kimyy/Observation/longhurst_v4_2010/Longhurst_1deg_mask.nc")


ModuleNotFoundError: No module named 'geopandas'